In [8]:
import torch
import torch.nn as nn

from src.cube import make_cube_mesh

from src.operators import get_gaussian

In [39]:
class Cube(nn.Module):
    def __init__(self, n, kernel=7, start=-0.5, end=0.5):
        super(Cube, self).__init__()        
        self.n = n
        self.params = nn.ParameterDict({
            'front': nn.Parameter(torch.zeros((3, n, n))),
            'back' : nn.Parameter(torch.zeros((3, n, n))),
            'left' : nn.Parameter(torch.zeros((3, n, n))),
            'right': nn.Parameter(torch.zeros((3, n, n))),
            'top'  : nn.Parameter(torch.zeros((3, n, n))),
            'down' : nn.Parameter(torch.zeros((3, n, n))),            
        })
        self.source = make_cube_mesh(n, start, end)
        self.gaussian = get_gaussian(kernel)
    
    def make_vert(self):
        return torch.cat([self.gaussian(p)[0].reshape(3, -1)  
                          for p in params.values()], dim=-1).t()        

    def forward(self):
        deform_verts = self.make_vert()
        new_src_mesh = self.source.offset_verts(deform_verts)
        return new_src_mesh
    
    def to(self, device):
        module = super(Cube, self).to(device)        
        module.source = self.source.to(device)        
        return module
    
    def export(self, f):
        mesh = self.forward().detach()
        save_obj(f, mesh.verts_packed(), mesh.faces_packed())    

In [2]:
g3 = get_gaussian(3)
g3

Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False, padding_mode=replicate)

In [3]:
g3(torch.randn(3, 3, 100, 100))

tensor([[[[ 7.9438e-01,  4.3005e-01, -6.6315e-01,  ..., -9.5278e-01,
           -7.8254e-01, -7.3293e-01],
          [-5.2570e-02, -2.0779e-01, -7.8589e-01,  ..., -6.2589e-01,
           -5.1360e-01, -3.3650e-01],
          [-8.4221e-02, -6.1115e-01, -1.0637e+00,  ..., -2.9301e-01,
           -2.2609e-01, -3.0271e-01],
          ...,
          [ 1.9079e-01,  6.9547e-03,  5.1614e-02,  ...,  4.0490e-01,
            1.5286e-01, -3.3568e-02],
          [-3.5926e-01, -1.6361e-01, -8.2602e-02,  ...,  6.7123e-01,
            2.1885e-01, -4.4470e-01],
          [-6.1303e-01,  2.5004e-01, -4.4083e-02,  ...,  6.9320e-01,
            5.4686e-01,  1.9382e-01]],

         [[ 2.2345e-01,  4.6967e-01,  3.8529e-01,  ..., -7.7210e-02,
            5.7891e-02, -2.0997e-01],
          [ 1.5053e-01,  3.1964e-01,  3.7400e-01,  ...,  2.9857e-01,
            1.8273e-01, -3.5422e-01],
          [-1.9012e-01, -2.0400e-01, -1.3082e-01,  ...,  4.9211e-01,
            1.7546e-01, -1.5087e-01],
          ...,
     

In [40]:
cube = Cube(11)
cube

Cube(
  (params): ParameterDict(
      (back): Parameter containing: [torch.FloatTensor of size 3x11x11]
      (down): Parameter containing: [torch.FloatTensor of size 3x11x11]
      (front): Parameter containing: [torch.FloatTensor of size 3x11x11]
      (left): Parameter containing: [torch.FloatTensor of size 3x11x11]
      (right): Parameter containing: [torch.FloatTensor of size 3x11x11]
      (top): Parameter containing: [torch.FloatTensor of size 3x11x11]
  )
  (gaussian): Conv2d(3, 3, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False, padding_mode=replicate)
)

In [41]:
cube.forward()

In [14]:
n =11
params = nn.ParameterDict({
            'front': nn.Parameter(torch.zeros((1, 3, n, n))),
            'back' : nn.Parameter(torch.zeros((1, 3, n, n))),
            'left' : nn.Parameter(torch.zeros((1, 3, n, n))),
            'right': nn.Parameter(torch.zeros((1, 3, n, n))),
            'top'  : nn.Parameter(torch.zeros((1, 3, n, n))),
            'down' : nn.Parameter(torch.zeros((1, 3, n, n))),            
        })
params

ParameterDict(
    (back): Parameter containing: [torch.FloatTensor of size 1x3x11x11]
    (down): Parameter containing: [torch.FloatTensor of size 1x3x11x11]
    (front): Parameter containing: [torch.FloatTensor of size 1x3x11x11]
    (left): Parameter containing: [torch.FloatTensor of size 1x3x11x11]
    (right): Parameter containing: [torch.FloatTensor of size 1x3x11x11]
    (top): Parameter containing: [torch.FloatTensor of size 1x3x11x11]
)

In [34]:
v = torch.cat([g3(p)[0].reshape(3, -1)  for p in params.values()], dim=-1).t()
v.shape

torch.Size([726, 3])

In [26]:
g3(params['front']).shape

torch.Size([1, 3, 11, 11])

In [27]:
g3(params['front'])[0].shape

torch.Size([3, 11, 11])

In [29]:
g3(params['front'])[0].reshape(3, -1).shape

torch.Size([3, 121])

In [48]:
if params['front'].data.grad:
    print(True)
else:
    print(False)

False


In [50]:
for p in params.values():
    print(p)
    p.data.grad = g3(p.data.grad)

Parameter containing:
tensor([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0

AttributeError: 'NoneType' object has no attribute 'dim'

In [46]:
p.data.grad

AttributeError: 'str' object has no attribute 'data'